In [11]:
import pandas as pd
import sqlite3
import json

#cargaré los datos de los archivos JSON

import pandas as pd
import sqlite3
import json

# Cargar los datos desde archivos JSON
customers = pd.read_json("sample_analytics.customers.json")
accounts = pd.read_json("sample_analytics.accounts.json")

# Cargar y expandir el JSON de transacciones (está anidado)
with open("sample_analytics.transactions.json") as f:
    raw_transactions = json.load(f)

# Extraer y normalizar las transacciones por cuenta
transactions_expanded = []
for entry in raw_transactions:
    account_id = entry['account_id']  # id de cuenta que agrupa cada set de transacciones
    for tx in entry['transactions']:
        tx_flat = tx.copy()
        tx_flat['account_id'] = account_id
        if isinstance(tx['date'], dict):  # revisar si la fecha está en formato {'$date': ...}
            tx_flat['date'] = tx['date']['$date']
        transactions_expanded.append(tx_flat)

# Crear DataFrame de transacciones y limpiar
transactions = pd.DataFrame(transactions_expanded)
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')  # convertir a datetime
transactions = transactions[['account_id', 'date', 'amount', 'transaction_code', 'symbol', 'price', 'total']]



In [12]:
#normalizaremos los clientes
customers['birthdate'] = customers['birthdate'].apply(lambda x: x['$date'] if isinstance(x, dict) else x)
customers['birthdate'] = pd.to_datetime(customers['birthdate'], errors='coerce')
customers = customers.explode("accounts").rename(columns={'accounts': 'account_id'})  # una fila por cuenta
customers_final = customers[['username', 'name', 'birthdate', 'account_id']]